In [ ]:
! pip install accelerate peft bitsandbytes transformers trl auto-gptq optimum

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer
import os

def finetune_deci_lm():
    data = load_dataset("yahma/alpaca-cleaned", split="train")
    data_df = data.to_pandas()
    data_df = data_df[:10000]
    data_df["text"] = data_df[["input", "instruction", "output"]].apply(lambda x: "###Human: " + x["instruction"] + "\n" + x["input"] + "\n###Assistant: " +x["output"], axis=1)
    print(data_df.iloc[0])
    data = Dataset.from_pandas(data_df)
    tokenizer = AutoTokenizer.from_pretrained("Deci/DeciLM-6b-instruct")
    tokenizer.pad_token = tokenizer.eos_token
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
    )
    model = AutoModelForCausalLM.from_pretrained(
        "Deci/DeciLM-6b-instruct", quantization_config=bnb_config, device_map="auto"
    )
    model.config.use_cache=False
    model.config.pretraining_tp=1
    peft_config = LoraConfig(
        r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM"
    )
    training_arguments = TrainingArguments(
        output_dir="deci-finetuned-alpaca-cleaned",
        per_device_train_batch_size=1,
        gradient_accumulation_steps=1,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=10,
        num_train_epochs=1,
        max_steps=1000,
        fp16=True,
        push_to_hub=True
    )
    trainer = SFTTrainer(
        model=model,
        train_dataset=data,
        peft_config=peft_config,
        dataset_text_field="text",
        args=training_arguments,
        tokenizer=tokenizer,
        packing=False,
        max_seq_length=512
    )
    trainer.train()
    trainer.push_to_hub()

if __name__ == "__main__":
    finetune_deci_lm()

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

output         1. Eat a balanced and nutritious diet: Make su...
instruction                 Give three tips for staying healthy.
input                                                           
text           ###Human: Give three tips for staying healthy....
Name: 0, dtype: object


A new version of the following files was downloaded from https://huggingface.co/Deci/DeciLM-6b-instruct:
- configuration_decilm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/Deci/DeciLM-6b-instruct:
- modeling_decilm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,2.173200
20,1.557400
30,1.199400
40,1.187700
50,1.385700
60,1.271700
70,1.435000
80,1.255900
90,1.220400
100,0.943900


Step,Training Loss
10,2.173200
20,1.557400
30,1.199400
40,1.187700
50,1.385700
60,1.271700
70,1.435000
80,1.255900
90,1.220400
100,0.943900


In [ ]:
! cp -r /content/deci-finetuned-alpaca-cleaned /content/drive/MyDrive/

# Inference

In [ ]:
from peft import AutoPeftModelForCausalLM
import torch
import os
model = AutoPeftModelForCausalLM.from_pretrained("deci-finetuned-alpaca-cleaned", device_map={"":0}, torch_dtype=torch.float16)
model = model.merge_and_unload()
model.push_to_hub("deci-finetuned-alpaca-cleaned")

Loading Deci/DeciLM-6b-instruct requires to execute some code in that repo, you can inspect the content of the repository at https://hf.co/Deci/DeciLM-6b-instruct. You can dismiss this prompt by passing `trust_remote_code=True`.
Do you accept? [y/N] y
Loading Deci/DeciLM-6b-instruct requires to execute some code in that repo, you can inspect the content of the repository at https://hf.co/Deci/DeciLM-6b-instruct. You can dismiss this prompt by passing `trust_remote_code=True`.
Do you accept? [y/N] y


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/1.47G [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.96G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Vasanth/deci-finetuned-alpaca-cleaned/commit/324b4ba7f1d340cd4cf4160a76e2879938d8da38', commit_message='Upload DeciLMForCausalLM', commit_description='', oid='324b4ba7f1d340cd4cf4160a76e2879938d8da38', pr_url=None, pr_revision=None, pr_num=None)

# Exporting to GPTQ

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig, BitsAndBytesConfig
import torch

model_id = "Vasanth/deci-finetuned-alpaca-cleaned"
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
    )
quantization_config = GPTQConfig(
    bits = 4,
    group_size = 64,
    desc_act = False,
    dataset = "c4"
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto", torch_dtype=torch.float16)
model

Loading Deci/DeciLM-6b-instruct requires to execute some code in that repo, you can inspect the content of the repository at https://hf.co/Deci/DeciLM-6b-instruct. You can dismiss this prompt by passing `trust_remote_code=True`.
Do you accept? [y/N] y
Loading Deci/DeciLM-6b-instruct requires to execute some code in that repo, you can inspect the content of the repository at https://hf.co/Deci/DeciLM-6b-instruct. You can dismiss this prompt by passing `trust_remote_code=True`.
Do you accept? [y/N] y


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

DeciLMForCausalLM(
  (model): DeciLMModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0): DeciLMDecoderLayer(
        (self_attn): DeciLMAttention(
          (q_proj): Linear4bit(
            in_features=4096, out_features=4096, bias=False
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (k_proj): Linear4bit(in_features=4096, out_features=512, bias=False)
          (v_proj): Linear4bit(
            in_features=4096, out_features=512, bias=False
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplac

In [ ]:
from time import perf_counter
from rich import print
from transformers import GenerationConfig

generation_config = GenerationConfig(
    penalty_alpha=0.6,
    do_sample = True,
    top_k=5,
    temperature=0.5,
    repetition_penalty=1.2,
    max_new_tokens=100,
    pad_token_id=tokenizer.eos_token_id
)
start_time = perf_counter()
inputs = tokenizer("""
###Instruction: I dropped my mobile phone in water. What to do?
###Response:""", return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
end_time = perf_counter()
output_time = end_time - start_time
print(f"Time taken for inference: {round(output_time,2)} seconds")

###Instruction: I dropped my mobile phone in water. What to do?
###Response: If you have a cellphone, it may be possible to salvage the device if you act quickly and take proper 
precautions. Here's what you should do:
1. Turn off your cell phone immediately after dropping it into water. This will prevent further damage from 
occurring.
2. Remove any accessories or cases that might be covering the charging port on the back of your phone. These could 
potentially block water from escaping when you plug your charger into the

Time taken for inference: 13.34 seconds